# Carregar dados históricos da Bovespa em dataframe para análise

In [1]:
import sys
from pandas import Series
import pandas as pd
from datetime import date
pd.__version__

'1.0.5'

In [2]:
import matplotlib as mat
mat.__version__

'3.2.2'

In [3]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [4]:
# Setando opção do Pandas para mostrar todas as linhas
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
# Setando separador de milhar
pd.options.display.float_format = '{:,.2f}'.format

In [6]:
# vr_investimento_padrao = 20000

In [7]:
def carrega_dados():
    layout = [2, 8, 2, 12, 3, 12, 10, 3, 4, 13, 13, 13, 13, 13, 13, 13, 5, 18, 18, 13, 1, 8, 7, 13, 12, 3]
    df = pd.read_fwf("arquivos/COTAHIST_A2021.ZIP", widths=layout, compression='zip',engine='c',
                    names=["TIPREG", "DTEXCH", "CODBDI", "CODNEG", "TPMERC", "NOMRES", "ESPECI", "PRAZOT", "MODREF", "PREABE", "PREMAX", "PREMIN", "PREMED", "PREULT", "PREOFC", "PREOFV", "TOTNEG", "QUATOT", "VOLTOT", "PREEXE", "INDOPC", "DATVEN", "FATCOT", "PTOEXE", "CODISI", "DISMES"])
    df = df.loc[df["CODBDI"] == 2]
    
    df02 = pd.DataFrame({
         "cdAcao":df["CODNEG"]
        ,"dtPregao":pd.to_datetime(df["DTEXCH"], format="%Y%m%d", errors="ignore")
        ,"vrFech":df["PREULT"] / 100
        ,"vrVolume":df["VOLTOT"] / 100
    #    ,"pc_variacao":((df["PREULT"] / df["PREULT"].shift(1)) -1) * 100
        ,"vrMaxDia":df["PREMAX"] / 100
        ,"vrMinDia":df["PREMIN"] / 100
    #    ,"pc_maximo_dia":
    #    ,"pc_minimo_dia":
        ,"vrAbert":df["PREABE"] / 100
    #    ,"pc_abertura":
    #    ,"ic_0_50_pc":
    #    ,"ic_1_00_pc":
    #    ,"ic_1_50_pc":
    #    ,"ic_2_00_pc":
    #    ,"ic_2_50_pc":
    #    ,"ic_3_00_pc":
    #    ,"vr_result_1_00_pc":
    #    ,"vr_result_1_50_pc":
    #    ,"vr_result_2_00_pc":
    #    ,"vr_result_2_50_pc":
    #    ,"vr_result_3_00_pc":
    })
    df02 = df02.sort_values(["cdAcao", "dtPregao"], ascending=True)
    
    df02["pcVar"], df02["pcMaxDia"], df02["pcMinDia"], df02["pcAbert"] = [
         ((df02.vrFech   / df02.vrFech.shift(1)) -1) *100
        ,((df02.vrMaxDia / df02.vrFech.shift(1)) -1) *100
        ,((df02.vrMinDia / df02.vrFech.shift(1)) -1) *100
        ,((df02.vrAbert  / df02.vrFech.shift(1)) -1) *100
        ]
    
    df02["ic05"], df02["ic10"], df02["ic15"], df02["ic20"], df02["ic25"], df02["ic30"] = [
         df02.apply(condicao05, axis=1)
        ,df02.apply(condicao10, axis=1)
        ,df02.apply(condicao15, axis=1)
        ,df02.apply(condicao20, axis=1)
        ,df02.apply(condicao25, axis=1)
        ,df02.apply(condicao30, axis=1)
        ]

    # Filtrando Ação
    # df02.loc[df02["cdAcao"]=="AMBP3"].sort_values(["cdAcao", "dtPregao"], ascending=False)
    
    return df02

In [8]:
def condicao05(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 0.5) else 0

In [9]:
def condicao10(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 1) else 0

In [10]:
def condicao15(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 1.5) else 0

In [11]:
def condicao20(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 2) else 0

In [12]:
def condicao25(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 2.5) else 0

In [13]:
def condicao30(dfTmp):
    return 1 if (dfTmp["pcMaxDia"] > 3) else 0

In [14]:
# Exclui header e trailer
# df = df.drop(df.index[[0, -1]])

In [ ]:
df = carrega_dados()

In [ ]:
df

In [ ]:
# Seleciona últimos 25 dias de cada ação
df25d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[24])].sort_values(["cdAcao", "dtPregao"], ascending=False)
df15d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[14])].sort_values(["cdAcao", "dtPregao"], ascending=False)
df20d = df.loc[df["dtPregao"]>=(df.dtPregao.drop_duplicates().sort_values(ascending=False).iloc[19])].sort_values(["cdAcao", "dtPregao"], ascending=False)

In [ ]:
df05 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30 = df25d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol = df25d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech = df25d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")

df05_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30_15d = df15d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol_15d = df15d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech_15d = df15d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")

df05_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>0.5).sum()).reset_index(name="0.5%")
df10_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.0).sum()).reset_index(name="resultado")
df15_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>1.5).sum()).reset_index(name="resultado")
df20_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.0).sum()).reset_index(name="resultado")
df25_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>2.5).sum()).reset_index(name="resultado")
df30_20d = df20d.groupby("cdAcao")["pcMaxDia"].apply(lambda x: (x>3.0).sum()).reset_index(name="resultado")
dfVol_20d = df20d.groupby("cdAcao")["vrVolume"].agg("mean").reset_index(name="vol")
dfVrFech_20d = df20d.groupby("cdAcao")["vrFech"].agg("mean").reset_index(name="vrFech")

In [ ]:
df05["1.0%"], df05["1.5%"], df05["2.0%"], df05["2.5%"], df05["3.0%"] , df05["AvgVol"], df05["AvgVrFech"] = [
    df10["resultado"],df15["resultado"],df20["resultado"],df25["resultado"],df30["resultado"],dfVol["vol"],dfVrFech["vrFech"]
]

df05_15d["1.0%"], df05_15d["1.5%"], df05_15d["2.0%"], df05_15d["2.5%"], df05_15d["3.0%"] , df05_15d["AvgVol"], df05_15d["AvgVrFech"] = [
    df10_15d["resultado"],df15_15d["resultado"],df20_15d["resultado"],df25_15d["resultado"],df30_15d["resultado"],dfVol_15d["vol"],dfVrFech_15d["vrFech"]
]

df05_20d["1.0%"], df05_20d["1.5%"], df05_20d["2.0%"], df05_20d["2.5%"], df05_20d["3.0%"] , df05_20d["AvgVol"], df05_20d["AvgVrFech"] = [
    df10_20d["resultado"],df15_20d["resultado"],df20_20d["resultado"],df25_20d["resultado"],df30_20d["resultado"],dfVol_20d["vol"],dfVrFech_20d["vrFech"]
]

In [ ]:
df05     =     df05.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_15d = df05_15d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)
df05_20d = df05_20d.reset_index(drop=True).sort_values(["1.0%", "1.5%", "2.0%", "2.5%", "3.0%"], ascending=False)

In [ ]:
pd.set_option('display.max_rows', None)
# df06 = df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= 20) & (df05["AvgVrFech"] > 5)]

In [ ]:
df25d['vrVolume'] = df25d['vrVolume'].astype(int)
df25d['vrVolume'] = df25d['vrVolume'].map('{:,}'.format)

### 25 dias, Order by 1%

In [ ]:
df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= 22) & (df05["AvgVrFech"] > 5)]

### 20 dias, Order by 1%

In [ ]:
df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["1.0%"] >= 18) & (df05_20d["AvgVrFech"] > 5)]

### 15 dias, Order by 1%

In [ ]:
df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["1.0%"] >= 15) & (df05_15d["AvgVrFech"] > 5)]

In [ ]:
### 25 dias, Order by 0.5%
# df05.loc[(df05["AvgVol"] > 6000000) & (df05["0.5%"] >= 23) & (df05["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 20 dias, Order by 0.5%
# df05_20d.loc[(df05_20d["AvgVol"] > 6000000) & (df05_20d["0.5%"] >= 19) & (df05_20d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)
### 15 dias, Order by 0.5%
# df05_15d.loc[(df05_15d["AvgVol"] > 6000000) & (df05_15d["0.5%"] >= 15) & (df05_15d["AvgVrFech"] > 5)].sort_values(["0.5%", "1.0%", "1.5%", "2.0%"], ascending=False)

### Consulta ação específica

In [ ]:
df25d.loc[(df25d["cdAcao"] == "BMOB3")].replace(0, "")

# Testes com botões e menus

In [ ]:
import ipywidgets as widgets

In [ ]:
from IPython.display import display, clear_output, Markdown

In [ ]:
button = widgets.Button(description='25')
out = widgets.Output()

def on_button_clicked(_):
      # "linking function with output"
      with out:
          # what happens when we press the button
          clear_output()
          print(df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= 25) & (df05["AvgVrFech"] > 5)])

In [ ]:
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button,out])

In [ ]:
menu = widgets.Dropdown(
    options=[25, 20, 15],
    value=25,
    description="Qt Dias:"
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        clear_output()
        display(menu)
#       print("changed to %s" % change['new'])
        print(df05.loc[(df05["AvgVol"] > 6000000) & (df05["1.0%"] >= change['new']) & (df05["AvgVrFech"] > 5)])
        
menu.observe(on_change)

display(menu)